<a href="https://colab.research.google.com/github/ez-zee/Annotation-Automation-Scripts-Python-/blob/main/cvat_to_roboflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
from roboflow import Roboflow

# === Параметры пользователя ===
VIDEO_DIR = r"C:\Users\felhasznaló\Desktop\CV datasets\VIDEO ANNOTATION"
EXTRACTED_FRAMES_DIR = r"C:\extracted_videoframes"
FRAME_INTERVAL = 10

ROBOFLOW_API_KEY = "Dpg4wVjxAMcUGN4QlzRY"
WORKSPACE_NAME = "cvat-vifgc"
PROJECT_NAME = "football-players-dvgnm"

# === Этап 1: Извлечение каждого 10-го кадра из всех видео ===
def extract_frames():
    os.makedirs(EXTRACTED_FRAMES_DIR, exist_ok=True)
    print(f"[INFO] Кадры будут сохранены в: {EXTRACTED_FRAMES_DIR}")

    video_files = [f for f in os.listdir(VIDEO_DIR) if f.lower().endswith(".mp4")]
    total_saved = 0

    for video_file in video_files:
        video_path = os.path.join(VIDEO_DIR, video_file)
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"[ERROR] Не удалось открыть видео: {video_path}")
            continue
        else:
            print(f"[DEBUG] Открыто: {video_path}")

        if not cap.isOpened():
            print(f"[WARN] Не удалось открыть {video_file}")
            continue

        frame_count = 0
        saved_count = 0
        base_name = os.path.splitext(video_file)[0]

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count % FRAME_INTERVAL == 0:
                filename = f"{base_name}_frame_{frame_count}.jpg"
                save_path = os.path.join(EXTRACTED_FRAMES_DIR, filename)
                success = cv2.imwrite(save_path, frame)
                if not success:
                    print(f"[ERROR] Не удалось сохранить кадр: {save_path}")
                else:
                    saved_count += 1

            frame_count += 1

        cap.release()
        print(f"[OK] {video_file}: сохранено {saved_count} кадров")
        total_saved += saved_count

    print(f"[DONE] Всего сохранено кадров: {total_saved}")

# === Этап 2: Загрузка кадров в Roboflow ===
def upload_to_roboflow():
    from glob import glob

    print("loading Roboflow workspace...")
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    workspace = rf.workspace(WORKSPACE_NAME)
    print("loading Roboflow project...")
    project = workspace.project(PROJECT_NAME)

    image_paths = glob(os.path.join(EXTRACTED_FRAMES_DIR, "*.jpg"))
    print(f"[INFO] Загружаем {len(image_paths)} изображений в Roboflow...")

    for i, image_path in enumerate(image_paths, 1):
        try:
            project.upload(image_path)
            print(f"[{i}/{len(image_paths)}] Загружено: {os.path.basename(image_path)}")
        except Exception as e:
            print(f"[ОШИБКА] {image_path}: {e}")

    print("[DONE] Все изображения загружены.")

# === Запуск ===
if __name__ == "__main__":
    extract_frames()
    upload_to_roboflow()